In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..', 'src')))
from funkwpap import *
import sympy as sp, pandas as pd, numpy as np, tqdm, time, sys, matplotlib.pyplot as plt, tqdm, math
from statistics import mean
from scipy.optimize import fsolve
import gurobipy as gp
from gurobipy import GRB


In [ ]:
# Define symbolic variable
x = sp.symbols('x')

# Create a Regulator with an emission cap
RegulatorETS = Regulator("ETS", permit_price=20.0, emission_cap=1000000)

# Define Countries with updated names and attributes
country1 = Country('Atlantis', size=50, regulator=RegulatorETS, GDP_per_capita=400, population=0, industry_percentage=30)
country2 = Country('Omashu', size=20, regulator=RegulatorETS, GDP_per_capita=350, population=0, industry_percentage=25)
country3 = Country('Hogsmeade', size=10, regulator=RegulatorETS, GDP_per_capita=450, population=0, industry_percentage=20)
country4 = Country('The_Court_of_Miracles', size=5, regulator=RegulatorETS, GDP_per_capita=300, population=0, industry_percentage=15)
country5 = Country('Lilipoupoli', size=2, regulator=RegulatorETS, GDP_per_capita=250, population=0, industry_percentage=10)

# Define Sectors with demand functions
sector1 = Sector('Steel', price_demand_function=200 - 0.1*x, free_emission_multiplier=1.3/1.6, regulator=RegulatorETS)
sector2 = Sector('Cement', price_demand_function=150 - 0.05*x, free_emission_multiplier=1/1.6, regulator=RegulatorETS)
sector3 = Sector('Paper', price_demand_function=100 - 0.02*x, free_emission_multiplier=0.9/1.6, regulator=RegulatorETS)
sector4 = Sector('Chemicals', price_demand_function=250 - 0.15*x, free_emission_multiplier=1.4/1.6, regulator=RegulatorETS)
sector5 = Sector('Automotive', price_demand_function=300 - 0.2*x, free_emission_multiplier=1.6/1.6, regulator=RegulatorETS)
sector6 = Sector('Textiles', price_demand_function=80 - 0.01*x, free_emission_multiplier=0.8/1.6, regulator=RegulatorETS)

# Create Firms with updated definitions (production_cost_function=0, abatement_cost_function as previous production_cost_function)
# Firms in Sector 1: Steel
firm_S1_F1 = Firm('S1_F1', sector1, country1, x*0, 2*x + 3*x**2 + x**3)
firm_S1_F2 = Firm('S1_F2', sector1, country1, x*0, 3*x + 2*x**2 + 2*x**3)
firm_S1_F3 = Firm('S1_F3', sector1, country2, x*0, 4*x + x**2 + 3*x**3)
firm_S1_F4 = Firm('S1_F4', sector1, country3, x*0, 2*x + 2*x**2 + 2*x**3)
firm_S1_F5 = Firm('S1_F5', sector1, country4, x*0, 3*x + 3*x**2 + x**3)

# Firms in Sector 2: Cement
firm_S2_F1 = Firm('S2_F1', sector2, country1, x*0, x + 2*x**2 + 3*x**3)
firm_S2_F2 = Firm('S2_F2', sector2, country1, x*0, 2*x + x**2 + 2*x**3)
firm_S2_F3 = Firm('S2_F3', sector2, country2, x*0, 3*x + x**2 + x**3)
firm_S2_F4 = Firm('S2_F4', sector2, country3, x*0, x + 3*x**2 + x**3)
firm_S2_F5 = Firm('S2_F5', sector2, country4, x*0, 2*x + 2*x**2 + x**3)

#Firms in Sector 3: Paper
firm_S3_F1 = Firm('S3_F1', sector3, country3, x*0, x + x**2 + x**3)
firm_S3_F2 = Firm('S3_F2', sector3, country4, x*0, x + 2*x**2 + x**3)
firm_S3_F3 = Firm('S3_F3', sector3, country5, x*0, x + x**2 + 2*x**3)
firm_S3_F4 = Firm('S3_F4', sector3, country2, x*0, 2*x + x**2 + x**3)
firm_S3_F5 = Firm('S3_F5', sector3, country1, x*0, x + 2*x**2 + 2*x**3)

# Firms in Sector 4: Chemicals
firm_S4_F1 = Firm('S4_F1', sector4, country1, x*0, 3*x + 4*x**2 + x**3)
firm_S4_F2 = Firm('S4_F2', sector4, country1, x*0, 4*x + 3*x**2 + 2*x**3)
firm_S4_F3 = Firm('S4_F3', sector4, country2, x*0, 2*x + 5*x**2 + x**3)
firm_S4_F4 = Firm('S4_F4', sector4, country3, x*0, 3*x + 3*x**2 + 3*x**3)
firm_S4_F5 = Firm('S4_F5', sector4, country4, x*0, 2*x + 2*x**2 + 4*x**3)

# Firms in Sector 5: Automotive
firm_S5_F1 = Firm('S5_F1', sector5, country1, x*0, 5*x + 4*x**2 + x**3)
firm_S5_F2 = Firm('S5_F2', sector5, country2, x*0, 4*x + 5*x**2 + 2*x**3)
firm_S5_F3 = Firm('S5_F3', sector5, country3, x*0, 3*x + 3*x**2 + 2*x**3)
firm_S5_F4 = Firm('S5_F4', sector5, country4, x*0, 2*x + 4*x**2 + 3*x**3)
firm_S5_F5 = Firm('S5_F5', sector5, country5, x*0, x + 3*x**2 + 4*x**3)

# Firms in Sector 6: Textiles
firm_S6_F1 = Firm('S6_F1', sector6, country5, x*0, x + x**2 + x**3)
firm_S6_F2 = Firm('S6_F2', sector6, country4, x*0, x + 2*x**2 + x**3)
firm_S6_F3 = Firm('S6_F3', sector6, country3, x*0, x + x**2 + 2*x**3)
firm_S6_F4 = Firm('S6_F4', sector6, country2, x*0, 2*x + x**2 + x**3)
firm_S6_F5 = Firm('S6_F5', sector6, country1, x*0, x + 2*x**2 + 2*x**3)


In [3]:
RegulatorETS.BAU_calculator(precision = 0.01, print_diff = True)
#print(RegulatorETS.BAU_emissions)
# 17638.87773  3750 for 2
RegulatorETS.emission_cap = RegulatorETS.BAU_emissions*0.8
# Optimal solution found for 2 sectors.
# Firm S1_F1 has output 451.62827788960453 and emission 446.7349764049005
# Firm S1_F2 has output 451.6282778896044 and emission 447.8616178523027
# Firm S1_F3 has output 451.6282778896045 and emission 448.41767394370135
# Firm S2_F1 has output 556.5131115584181 and emission 553.3583289997191
# Firm S2_F2 has output 556.513111558418 and emission 552.5696348002654
# Firm S2_F3 has output 556.5131115584182 and emission 551.0577679991109
# Permit price: 103.19300716884365

Set parameter Username


Academic license - for non-commercial use only - expires 2025-08-14
Max diff: 0.01, permit price = 20.0, cap = 1000000, second_stage = False, a = 1.000000

In [4]:
for sector in RegulatorETS.sector_registry.values():
    sector.free_emission_multiplier = 0
RegulatorETS.optimization_with_positive_constraints_ab(print_output= True, gurobi_print=False)
RegulatorETS.equilibrium_tester(full_output=True)

Optimal solution found
Firm S1_F1 has output 297.03207404856636 and emission 295.2764279917792
Firm S1_F2 has output 297.0320740485663 and emission 295.56506379258457
Firm S1_F3 has output 297.03207404856624 and emission 295.7332258741355
Firm S1_F4 has output 297.03207404856624 and emission 295.5193565412523
Firm S1_F5 has output 297.03207404856624 and emission 295.33758859546884
Firm S2_F1 has output 427.3974814304658 and emission 426.0840098169069
Firm S2_F2 has output 427.39748143046586 and emission 425.7408076536275
Firm S2_F3 has output 427.39748143046575 and emission 425.20665887161164
Firm S2_F4 has output 427.3974814304658 and emission 425.5820030477358
Firm S2_F5 has output 427.39748143046586 and emission 425.4112195955081
Firm S3_F1 has output 651.827036909498 and emission 649.5074417412069
Firm S3_F2 has output 651.8270369094981 and emission 649.7786781678003
Firm S3_F3 has output 651.8270369094981 and emission 650.1252183502163
Firm S3_F4 has output 651.827036909498 and em

(True, 2.58992827184557e-12)

In [5]:
# Calculate population of each country based on GDP per capita and percentafe of industry in the gdp and the economic profit of the firms in the country.

for country in RegulatorETS.country_registry.values():
    country.GDP = 0
for firm in RegulatorETS.firm_registry.values():
    firm.calculate_profit_components()
    firm.country.GDP += firm.sales
for country in RegulatorETS.country_registry.values():
    country.GDP = country.GDP/country.industry_percentage*100
    country.population = math.floor(country.GDP/country.GDP_per_capita)
    print(f"Population of {country.name} is {country.population}, and GDP is {country.GDP}")

Population of Atlantis is 1390, and GDP is 556171.230334218
Population of Omashu is 1347, and GDP is 471792.894053352
Population of Hogsmeade is 1310, and GDP is 589741.117566691
Population of The_Court_of_Miracles is 2621, and GDP is 786321.490088921
Population of Lilipoupoli is 2761, and GDP is 690450.779264107


In [6]:
# # To recap.
# for country in RegulatorETS.country_registry.values():
#     print(f"Country: {country.name}, Size: {country.size}, GDP per capita: {country.GDP_per_capita}, Population: {country.population}, Industry percentage: {country.industry_percentage}, GDP: {country.GDP}")
#     print(country.firms)
# for firm in RegulatorETS.firm_registry.values():
#     print(f"Firm {firm.name} in sector {firm.sector.name} in country {firm.country.name} has the following attributes:")
#     print(f"Production cost function: {firm.production_cost_function}, Abatement cost function: {firm.abatement_cost_function}, Actual output: {firm.actual_output}, Emission: {firm.emission}, Profit: {firm.profit}, BAU output: {firm.BAU_output}, BAU emission: {firm.BAU_emission}, BAU profit: {firm.BAU_profit}, Regulator: {firm.regulator}, Abatement: {firm.abatement}, Permits costs: {firm.permits_costs}, Sales: {firm.sales}, Permits used: {firm.permits_used}, Free permits: {firm.free_permits}, Permits bought: {firm.permits_bought}, Exp7 free allocation: {firm.exp7_free_allocation}")
# for sector in RegulatorETS.sector_registry.values():
#     print(f"Sector {sector.name}, from Regulator {sector.regulator} has the following attributes:")
#     print(f"ID: {sector.id}, Price demand function: {sector.price_demand_function}, Free emission multiplier: {sector.free_emission_multiplier}, Firms: {sector.firms}")

In [7]:
aggregate_data = {}
for country in RegulatorETS.country_registry.values():
    for sector in RegulatorETS.sector_registry.values():
        # Filter firms belonging to the current country and sector
        firms = [firm for firm in RegulatorETS.firm_registry.values()
                 if firm.country == country and firm.sector == sector]
        if firms:
            total_sales = sum(firm.sales for firm in firms)
            total_emissions = sum(firm.emission for firm in firms)
            aggregate_data[(country.name, sector.name)] = {
                'sales': total_sales,
                'emissions': total_emissions
            }

aggregate_data

{('Atlantis', 'Steel'): {'sales': 30584.7766058335,
  'emissions': 590.8414917843638},
 ('Atlantis', 'Cement'): {'sales': 36884.9408625871,
  'emissions': 851.8248174705344},
 ('Atlantis', 'Paper'): {'sales': 22694.8550863282,
  'emissions': 650.2697168179749},
 ('Atlantis', 'Chemicals'): {'sales': 30336.5737054342,
  'emissions': 504.35945552259636},
 ('Atlantis', 'Automotive'): {'sales': 15800.6805720617,
  'emissions': 230.46767828258538},
 ('Atlantis', 'Textiles'): {'sales': 30549.5422680208,
  'emissions': 968.7634203941399},
 ('Omashu', 'Steel'): {'sales': 15292.3883029168,
  'emissions': 295.7332258741355},
 ('Omashu', 'Cement'): {'sales': 18442.4704312935,
  'emissions': 425.20665887161164},
 ('Omashu', 'Paper'): {'sales': 22694.8550863282,
  'emissions': 649.5710274108761},
 ('Omashu', 'Chemicals'): {'sales': 15168.2868527171,
  'emissions': 252.18233672522993},
 ('Omashu', 'Automotive'): {'sales': 15800.6805720617,
  'emissions': 230.7701389975983},
 ('Omashu', 'Textiles'): {

In [8]:
# Initialize the Gurobi model
model = gp.Model("EU_ETL_Optimization")

# Extract country and sector names
countries = [country.name for country in RegulatorETS.country_registry.values()]
sectors = [sector.name for sector in RegulatorETS.sector_registry.values()]

# Decision variables: v_{i,j}
v = {}
for country in countries:
    for sector in sectors:
        if (country, sector) in aggregate_data:
            v[(country, sector)] = model.addVar(name=f"v_{country}_{sector}", lb=0, ub=1, vtype=GRB.CONTINUOUS)

# Aggregate allocation variables: v_i for countries and v_j for sectors
v_i = {}
for country in countries:
    v_i[country] = model.addVar(name=f"v_{country}_aggregate", lb=0, ub=1, vtype=GRB.CONTINUOUS)

v_j = {}
for sector in sectors:
    v_j[sector] = model.addVar(name=f"v_{sector}_aggregate", lb=0, ub=1, vtype=GRB.CONTINUOUS)


In [9]:
# Define the objective function
objective = gp.LinExpr()
for (country, sector), data in aggregate_data.items():
    sales = data['sales']
    emissions = data['emissions']
    if emissions > 0:
        efficiency = sales / emissions
    else:
        efficiency = 0  # Handle division by zero if necessary
    objective += v[(country, sector)] * efficiency

model.setObjective(objective, GRB.MAXIMIZE)

In [10]:
# Total free allocation to distribute (normalized to 1)
total_free_allocation = 1

# Total Cap Constraint
model.addConstr(gp.quicksum(v.values()) == total_free_allocation, "Total_Cap_Constraint")

<gurobi.Constr *Awaiting Model Update*>

In [11]:
# Country Allocation Constraints
for country in countries:
    model.addConstr(
        gp.quicksum(v[(country, sector)] for sector in sectors if (country, sector) in v) == v_i[country],
        f"Country_Alloc_{country}"
    )

# Sector Allocation Constraints
for sector in sectors:
    model.addConstr(
        gp.quicksum(v[(country, sector)] for country in countries if (country, sector) in v) == v_j[sector],
        f"Sector_Alloc_{sector}"
    )



Here’s the code to add the described constraints, ensuring each \( v_{i,j} \) is within the bounds based on the emission percentages:

```python
# Define coefficients
a1 = 0.5
a2 = 1.5

# Calculate the sum of emissions
sum_of_emissions = sum(firm.emission for firm in RegulatorETS.firm_registry.values())

# Add constraints for each (country, sector) pair
for (country, sector) in v.keys():
    # Calculate emission_{i,j}
    emission_ij = sum(
        firm.emission
        for firm in RegulatorETS.firm_registry.values()
        if firm.country.name == country and firm.sector.name == sector
    )
    
    # Add the constraints
    model.addConstr(
        a1 * (emission_ij / sum_of_emissions) <= v[(country, sector)],
        f"Emission_LowerBound_{country}_{sector}"
    )
    model.addConstr(
        v[(country, sector)] <= a2 * (emission_ij / sum_of_emissions),
        f"Emission_UpperBound_{country}_{sector}"
    )
```

### Explanation:
1. **Coefficient Definition**:
   - \( a1 \) and \( a2 \) are set to 0.5 and 1.5 respectively.

2. **Sum of Emissions**:
   - The total emissions are computed by summing up the `emission` attribute of all firms.

3. **Emissions per (Country, Sector)**:
   - For each country-sector pair, we sum the emissions of firms that belong to the corresponding country and sector.

4. **Constraints**:
   - Two constraints are added:
     - \( a1 \times \frac{\text{emission}_{i,j}}{\text{sum\_of\_emissions}} \leq v_{i,j} \)
     - \( v_{i,j} \leq a2 \times \frac{\text{emission}_{i,j}}{\text{sum\_of\_emissions}} \)

This ensures \( v_{i,j} \) stays within the bounds relative to the emissions of that country-sector pair.


In [12]:
# Define coefficients
a1 = 0.5
a2 = 1.5

# Calculate the sum of emissions
sum_of_emissions = sum(firm.emission for firm in RegulatorETS.firm_registry.values())

# Add constraints for each (country, sector) pair
for (country, sector) in v.keys():
    # Calculate emission_{i,j}
    emission_ij = sum(
        firm.emission
        for firm in RegulatorETS.firm_registry.values()
        if firm.country.name == country and firm.sector.name == sector
    )
    
    # Add the constraints
    model.addConstr(
        a1 * (emission_ij / sum_of_emissions) <= v[(country, sector)],
        f"Emission_LowerBound_{country}_{sector}"
    )
    model.addConstr(
        v[(country, sector)] <= a2 * (emission_ij / sum_of_emissions),
        f"Emission_UpperBound_{country}_{sector}"
    )

Certainly! Below is the additional code that adds constraints based on the inverse GDP per capita for each country-sector pair. These constraints ensure that each \( v_{i,j} \) is within a relaxed range determined by the inverse GDP per capita proportions.

```python
# Define additional coefficients
a3 = 0.8  # Example value for lower multiplier
a4 = 1.2  # Example value for upper multiplier

# Calculate the sum of inverse GDP per capita
# inverse_GDP_per_capita_{i,j} = population / sum_of_sales_of_firms in country i and sector j
inverse_gdp = {}
sum_inverse_gdp = 0

for (country, sector) in v.keys():
    # Get the population of the country
    population = next(country_obj.population for country_obj in RegulatorETS.country_registry.values() if country_obj.name == country)
    
    # Calculate the sum of sales for firms in this country and sector
    sum_sales = sum(
        firm.sales
        for firm in RegulatorETS.firm_registry.values()
        if firm.country.name == country and firm.sector.name == sector
    )
    
    # Avoid division by zero
    if sum_sales > 0:
        inverse_gdp_ij = population / sum_sales
    else:
        inverse_gdp_ij = 0
    
    inverse_gdp[(country, sector)] = inverse_gdp_ij
    sum_inverse_gdp += inverse_gdp_ij

# Add constraints based on inverse GDP per capita
for (country, sector) in v.keys():
    if sum_inverse_gdp > 0:
        proportion = inverse_gdp[(country, sector)] / sum_inverse_gdp
    else:
        proportion = 0  # Handle case where sum_inverse_gdp is zero
    
    # Add the relaxed constraints
    model.addConstr(
        a3 * proportion <= v[(country, sector)],
        f"InverseGDP_LowerBound_{country}_{sector}"
    )
    model.addConstr(
        v[(country, sector)] <= a4 * proportion,
        f"InverseGDP_UpperBound_{country}_{sector}"
    )
```

### Explanation:

1. **Coefficient Definition**:
   - `a3` and `a4` are set to `0.8` and `1.2` respectively as examples. You can adjust these values to make the constraints more or less relaxed based on your requirements.

2. **Inverse GDP per Capita Calculation**:
   - For each country-sector pair, calculate the inverse GDP per capita as:
     \[
     \text{inverse\_GDP}_{i,j} = \frac{\text{population of country } i}{\sum \text{sales of firms in country } i \text{ and sector } j}
     \]
   - Ensure that you handle cases where the sum of sales is zero to avoid division by zero errors.

3. **Sum of Inverse GDP per Capita**:
   - Compute the total sum of all inverse GDP per capita values across all country-sector pairs. This is used to normalize the proportions.

4. **Adding the Constraints**:
   - For each \( v_{i,j} \), calculate its proportion relative to the total inverse GDP per capita:
     \[
     \text{proportion}_{i,j} = \frac{\text{inverse\_GDP}_{i,j}}{\sum \text{inverse\_GDP}}
     \]
   - Add two constraints to ensure that \( v_{i,j} \) is within the relaxed bounds:
     \[
     a3 \times \text{proportion}_{i,j} \leq v_{i,j} \leq a4 \times \text{proportion}_{i,j}
     \]
   - These constraints are named uniquely for each country-sector pair for clarity.

### Notes:

- **Adjusting Coefficients**: The values of `a3` and `a4` should be chosen based on how strictly you want to enforce the inverse GDP per capita allocation. Values closer to `1` make the constraints tighter, while values further away make them more relaxed.
  
- **Handling Zero Sales**: If a country-sector pair has zero sales, the inverse GDP per capita is set to `0`, effectively removing any constraints on \( v_{i,j} \) for that pair based on this criterion. Ensure this aligns with your allocation strategy.

- **Integration with Existing Constraints**: These new constraints will work in conjunction with the previously defined emission-based constraints, ensuring that allocations respect both emission proportions and inverse GDP per capita considerations.

Make sure to integrate this code **after** the existing constraints and **before** the model optimization (`model.optimize()`) to ensure all constraints are considered during optimization.


In [13]:
# Define additional coefficients
a3 = 0.25  # Example value for lower multiplier
a4 = 4  # Example value for upper multiplier

# Calculate the sum of inverse GDP per capita
# inverse_GDP_per_capita_{i,j} = population / sum_of_sales_of_firms in country i and sector j
inverse_gdp = {}
sum_inverse_gdp = 0

for (country, sector) in v.keys():
    # Get the population of the country
    population = next(country_obj.population for country_obj in RegulatorETS.country_registry.values() if country_obj.name == country)
    
    # Calculate the sum of sales for firms in this country and sector
    sum_sales = sum(
        firm.sales
        for firm in RegulatorETS.firm_registry.values()
        if firm.country.name == country and firm.sector.name == sector
    )
    
    # Avoid division by zero
    if sum_sales > 0:
        inverse_gdp_ij = population / sum_sales
    else:
        inverse_gdp_ij = 0
    
    inverse_gdp[(country, sector)] = inverse_gdp_ij
    sum_inverse_gdp += inverse_gdp_ij

# Add constraints based on inverse GDP per capita
for (country, sector) in v.keys():
    if sum_inverse_gdp > 0:
        proportion = inverse_gdp[(country, sector)] / sum_inverse_gdp
    else:
        proportion = 0  # Handle case where sum_inverse_gdp is zero
    
    # Add the relaxed constraints
    model.addConstr(
        a3 * proportion <= v[(country, sector)],
        f"InverseGDP_LowerBound_{country}_{sector}"
    )
    model.addConstr(
        v[(country, sector)] <= a4 * proportion,
        f"InverseGDP_UpperBound_{country}_{sector}"
    )


In [14]:
# # Example: Fairness based on aggregate GDP per country
# # First, calculate total GDP
# total_GDP = sum(country.GDP for country in RegulatorETS.country_registry.values())

# # Add fairness constraints
# for country in countries:
#     fair_share = country_GDP = next(c.GDP for c in RegulatorETS.country_registry.values() if c.name == country) / total_GDP
#     model.addConstr(
#         v_i[country] >= (fair_share - alpha_fairness),
#         f"Fairness_Lower_{country}"
#     )
#     model.addConstr(
#         v_i[country] <= (fair_share + alpha_fairness),
#         f"Fairness_Upper_{country}"
#     )


In [15]:
# Optimize the model
model.optimize()
total_free_allocation = RegulatorETS.emission_cap*0.4
# Check if the optimization was successful
if model.status == GRB.OPTIMAL:
    print("Optimization was successful.")
    
    # Assign allocations to firms
    for (country, sector), var in v.items():
        allocation_percentage = var.X  # Optimized v_{i,j}
        firms = [firm for firm in RegulatorETS.firm_registry.values()
                 if firm.country.name == country and firm.sector.name == sector]
        if firms:
            total_production = sum(firm.actual_output for firm in firms)
            for firm in firms:
                # Calculate the firm's share based on production
                if total_production > 0:
                    firm_share = firm.actual_output / total_production
                else:
                    firm_share = 1 / len(firms)  # Equal split if no production data
                firm.exp7_free_allocation = allocation_percentage * firm_share * total_free_allocation
                print(f"Assigned {firm.exp7_free_allocation:.4f} free permits to {firm.name}")
else:
    print("Optimization was not successful. Status code:", model.status)


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i9-9900 CPU @ 3.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 120 rows, 38 columns and 200 nonzeros
Model fingerprint: 0x52073ba2
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 7e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e-03, 1e+00]
Presolve removed 119 rows and 11 columns
Presolve time: 0.01s
Presolved: 1 rows, 27 columns, 27 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.4977296e+01   4.477848e-01   0.000000e+00      0s
       1    4.6148325e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.614832463e+01
Optimization was successful.
Assigned 177.2524 free permits to S1_F1
Assigned 177.2524 free permits to S1_F2
Ass

In [16]:
# Optionally, you can print the allocations
print("\nOptimized Allocations (v_{i,j}):")
for (country, sector), var in v.items():
    print(f"v_{country}_{sector} = {var.X:.4f}")

print("\nAggregate Allocations per Country (v_i):")
for country, var in v_i.items():
    print(f"v_{country} = {var.X:.4f}")

print("\nAggregate Allocations per Sector (v_j):")
for sector, var in v_j.items():
    print(f"v_{sector} = {var.X:.4f}")


Optimized Allocations (v_{i,j}):
v_Atlantis_Steel = 0.0628
v_Atlantis_Cement = 0.0621
v_Atlantis_Paper = 0.0230
v_Atlantis_Chemicals = 0.0536
v_Atlantis_Automotive = 0.0245
v_Atlantis_Textiles = 0.0343
v_Omashu_Steel = 0.0314
v_Omashu_Cement = 0.0452
v_Omashu_Paper = 0.0690
v_Omashu_Chemicals = 0.0268
v_Omashu_Automotive = 0.0245
v_Omashu_Textiles = 0.0343
v_Hogsmeade_Steel = 0.0314
v_Hogsmeade_Cement = 0.0452
v_Hogsmeade_Paper = 0.0690
v_Hogsmeade_Chemicals = 0.0268
v_Hogsmeade_Automotive = 0.0245
v_Hogsmeade_Textiles = 0.0343
v_The_Court_of_Miracles_Steel = 0.0314
v_The_Court_of_Miracles_Cement = 0.0452
v_The_Court_of_Miracles_Paper = 0.0328
v_The_Court_of_Miracles_Chemicals = 0.0268
v_The_Court_of_Miracles_Automotive = 0.0245
v_The_Court_of_Miracles_Textiles = 0.0343
v_Lilipoupoli_Paper = 0.0230
v_Lilipoupoli_Automotive = 0.0245
v_Lilipoupoli_Textiles = 0.0343

Aggregate Allocations per Country (v_i):
v_Atlantis = 0.2603
v_Omashu = 0.2313
v_Hogsmeade = 0.2314
v_The_Court_of_Miracle

In [17]:
RegulatorETS.exp7_optimization_with_positive_constraints_ab(print_output= True, gurobi_print=False)

Optimal solution found
Firm S1_F1 has output 297.03207404856636 and emission 295.2764279917792
Firm S1_F2 has output 297.0320740485663 and emission 295.56506379258457
Firm S1_F3 has output 297.03207404856624 and emission 295.7332258741355
Firm S1_F4 has output 297.03207404856624 and emission 295.5193565412523
Firm S1_F5 has output 297.03207404856624 and emission 295.33758859546884
Firm S2_F1 has output 427.3974814304658 and emission 426.0840098169069
Firm S2_F2 has output 427.39748143046586 and emission 425.7408076536275
Firm S2_F3 has output 427.39748143046575 and emission 425.20665887161164
Firm S2_F4 has output 427.3974814304658 and emission 425.5820030477358
Firm S2_F5 has output 427.39748143046586 and emission 425.4112195955081
Firm S3_F1 has output 651.827036909498 and emission 649.5074417412069
Firm S3_F2 has output 651.8270369094981 and emission 649.7786781678003
Firm S3_F3 has output 651.8270369094981 and emission 650.1252183502163
Firm S3_F4 has output 651.827036909498 and em

<gurobi.Model MIP instance positive_constraints_ab: 91 constrs, 61 vars, Parameter changes: Username=(user-defined), OutputFlag=0>

In [18]:
consumer_surplus = 0
for sector in RegulatorETS.sector_registry.values():
    consumer_surplus += sector.get_consumer_surplus()
print(consumer_surplus)
# 703313.055279207 for no fairness

703313.055279207


In [19]:
for firm in RegulatorETS.firm_registry.values():
    firm.exp7_calculate_profit_components()


In [20]:
# =======================
# 1. Configuration Variables
# =======================
RUN_NAME = 'run_case_2'  # Unique identifier for the current run
BASE_PATH = 'C:/Users/Kostas/Documents/GitHub/Diplomatiki_kwpap_step_1/2024 Project v2/data/exp7'  # Fixed base path for data storage

# =======================
# 5. Data Saving
# =======================

def save_data(regulator, run_name, base_path):
    """
    Saves the data from the regulator object into CSV files.

    Parameters:
    - regulator (Regulator): The regulator instance containing all data.
    - run_name (str): Unique identifier for the current run.
    - base_path (str): Fixed base directory for data storage.
    """
    # Define run-specific paths
    run_path = os.path.join(base_path, run_name)
    data_dir = os.path.join(run_path, 'data')
    summaries_dir = os.path.join(run_path, 'summaries')

    # Create directories if they don't exist
    os.makedirs(data_dir, exist_ok=True)
    os.makedirs(summaries_dir, exist_ok=True)

    # 1. Extract Regulator Data
    regulator_data = {
        'id': [regulator.id],
        'name': [regulator.name],
        'permit_price': [regulator.permit_price],
        'emission_cap': [regulator.emission_cap],
        'BAU_emissions': [regulator.BAU_emissions],
        'BAU_profit': [regulator.BAU_profit]
    }

    df_regulator = pd.DataFrame(regulator_data)

    # Save Regulator DataFrame to CSV
    regulator_csv = os.path.join(data_dir, f'{run_name}_regulator.csv')
    df_regulator.to_csv(regulator_csv, index=False)
    print(f"Regulator data saved to {regulator_csv}.")

    # 2. Extract Countries Data
    countries = regulator.country_registry.values()
    country_list = []
    for country in countries:
        country_list.append({
            'id': country.id,
            'name': country.name,
            'size': country.size,
            'GDP_per_capita': country.GDP_per_capita,
            'population': country.population,
            'industry_percentage': country.industry_percentage,
            'GDP': country.GDP
        })

    df_countries = pd.DataFrame(country_list)

    # Save Countries DataFrame to CSV
    countries_csv = os.path.join(data_dir, f'{run_name}_countries.csv')
    df_countries.to_csv(countries_csv, index=False)
    print(f"Countries data saved to {countries_csv}.")

    # 3. Extract Sectors Data
    sectors = regulator.sector_registry.values()
    sector_list = []
    for sector in sectors:
        sector_list.append({
            'id': sector.id,
            'name': sector.name,
            'price_demand_function': str(sector.price_demand_function),
            'free_emission_multiplier': sector.free_emission_multiplier,
            'consumer_surplus': sector.consumer_surplus  # Include consumer_surplus
        })

    df_sectors = pd.DataFrame(sector_list)

    # Save Sectors DataFrame to CSV
    sectors_csv = os.path.join(data_dir, f'{run_name}_sectors.csv')
    df_sectors.to_csv(sectors_csv, index=False)
    print(f"Sectors data saved to {sectors_csv}.")

    # 4. Extract Firms Data
    firm_list = []
    for sector in sectors:
        for firm in sector.firms:
            firm_list.append({
                'id': firm.id,
                'name': firm.name,
                'sector_id': firm.sector.id,
                'country_id': firm.country.id,
                'production_cost_function': str(firm.production_cost_function),
                'abatement_cost_function': str(firm.abatement_cost_function),
                'actual_output': firm.actual_output,
                'emission': firm.emission,
                'profit': firm.profit,
                'BAU_output': firm.BAU_output,
                'BAU_emission': firm.BAU_emission,
                'BAU_profit': firm.BAU_profit,
                'exp7_free_allocation': firm.exp7_free_allocation,
                'abatement': firm.abatement,
                'permits_costs': firm.permits_costs,
                'sales': firm.sales,
                'permits_used': firm.permits_used,
                'free_permits': firm.free_permits,
                'permits_bought': firm.permits_bought
            })

    df_firms = pd.DataFrame(firm_list)

    # Save Firms DataFrame to CSV
    firms_csv = os.path.join(data_dir, f'{run_name}_firms.csv')
    df_firms.to_csv(firms_csv, index=False)
    print(f"Firms data saved to {firms_csv}.")

    # 5. Save Summary Statistics
    # Regulator Summary
    regulator_summary_csv = os.path.join(summaries_dir, f'{run_name}_summary_regulator.csv')
    df_regulator.describe().to_csv(regulator_summary_csv)
    print(f"Regulator summary saved to {regulator_summary_csv}.")

    # Sectors Summary
    sectors_summary_csv = os.path.join(summaries_dir, f'{run_name}_summary_sectors.csv')
    df_sectors.describe(include='all').to_csv(sectors_summary_csv)
    print(f"Sectors summary saved to {sectors_summary_csv}.")

    # Countries Summary
    countries_summary_csv = os.path.join(summaries_dir, f'{run_name}_summary_countries.csv')
    df_countries.describe().to_csv(countries_summary_csv)
    print(f"Countries summary saved to {countries_summary_csv}.")

    # Firms Summary
    firms_summary_csv = os.path.join(summaries_dir, f'{run_name}_summary_firms.csv')
    df_firms.describe(include='all').to_csv(firms_summary_csv)
    print(f"Firms summary saved to {firms_summary_csv}.")

    print("\nAll data and summaries have been saved successfully.")

# =======================
# 6. Execute Data Saving
# =======================
save_data(RegulatorETS, RUN_NAME, BASE_PATH)

Regulator data saved to C:/Users/Kostas/Documents/GitHub/Diplomatiki_kwpap_step_1/2024 Project v2/data/exp7\run_case_2\data\run_case_2_regulator.csv.
Countries data saved to C:/Users/Kostas/Documents/GitHub/Diplomatiki_kwpap_step_1/2024 Project v2/data/exp7\run_case_2\data\run_case_2_countries.csv.
Sectors data saved to C:/Users/Kostas/Documents/GitHub/Diplomatiki_kwpap_step_1/2024 Project v2/data/exp7\run_case_2\data\run_case_2_sectors.csv.
Firms data saved to C:/Users/Kostas/Documents/GitHub/Diplomatiki_kwpap_step_1/2024 Project v2/data/exp7\run_case_2\data\run_case_2_firms.csv.
Regulator summary saved to C:/Users/Kostas/Documents/GitHub/Diplomatiki_kwpap_step_1/2024 Project v2/data/exp7\run_case_2\summaries\run_case_2_summary_regulator.csv.
Sectors summary saved to C:/Users/Kostas/Documents/GitHub/Diplomatiki_kwpap_step_1/2024 Project v2/data/exp7\run_case_2\summaries\run_case_2_summary_sectors.csv.
Countries summary saved to C:/Users/Kostas/Documents/GitHub/Diplomatiki_kwpap_step_1